# Caltech Machine Learning Homework # 7

In [1]:
import numpy as np
from sklearn.linear_model import Perceptron
import random
import math
import matplotlib.pyplot as plt
from typing import List
from itertools import product
import scipy.special
from scipy import optimize
import scipy.optimize as spo
from sympy import Symbol, Derivative
import functools
from sklearn import svm

def dbg():
    import pdb; pdb.set_trace()

Instructions: https://work.caltech.edu/homework/hw7.pdf

Answers: http://work.caltech.edu/homework/hw7_sol.pdf

## Validation

![](imgs/val1.png)

In [23]:
data = np.loadtxt('data/hw7/in.dta.txt')

N_train = 25
X_train = data[:N_train,:-1]
Y_train = data[:N_train,2]

N_val = 10
X_val = data[-N_val:,:-1]
Y_val = data[-N_val:,2]

def phi(X, k):
    assert(X.shape == (2,))
    x1, x2 = X
    return np.array([1, x1, x2, x1 * x1, x2 * x2, x1 * x2, np.abs(x1-x2), np.abs(x1+x2)])[:k+1]

for k in range(3, 8):
    # Non-Linear Transformation
    Z_train = np.apply_along_axis(phi, 1, X_train, k)
    Z_val = np.apply_along_axis(phi, 1, X_val, k)

    # Linear Regression
    X_dagger = np.dot(np.linalg.inv(np.dot(Z_train.T, Z_train)), Z_train.T)
    W = np.dot(X_dagger, Y_train)

    # Validation Error
    preds_val = np.sign(np.dot(Z_val, W))
    E_VAL = sum(preds_val != Y_val) / N_val

    print(f"E_VAL for k={k} is {E_VAL}")


E_VAL for k=3 is 0.3
E_VAL for k=4 is 0.5
E_VAL for k=5 is 0.2
E_VAL for k=6 is 0.0
E_VAL for k=7 is 0.1


So the smallest classification error on the validation set occurs for `k=6`, **[d]**.

![](imgs/val2.png)

In [24]:
test = np.loadtxt('data/hw7/out.dta.txt')

X_test = test[:,:-1]
Y_test = test[:,2]
N_test = Y_test.shape[0]

for k in range(3, 8):
    # Non-Linear Transformation
    Z_train = np.apply_along_axis(phi, 1, X_train, k)
    Z_test = np.apply_along_axis(phi, 1, X_test, k)

    # Linear Regression
    X_dagger = np.dot(np.linalg.inv(np.dot(Z_train.T, Z_train)), Z_train.T)
    W = np.dot(X_dagger, Y_train)

    # Out of Sample Error
    preds_test = np.sign(np.dot(Z_test, W))
    E_OUT = sum(preds_test != Y_test) / N_test

    print(f"E_OUT for k={k} is {E_OUT}")

E_OUT for k=3 is 0.42
E_OUT for k=4 is 0.416
E_OUT for k=5 is 0.188
E_OUT for k=6 is 0.084
E_OUT for k=7 is 0.072


So the smalles out-of-sample error on the test set was almost predicted correctly, but not quite. The answer is **[e]**

![](imgs/val3.png)

I'm expecting the validation set predictions to be wayyy better now, and a simpler model to "win" since we have less examples. Let's see :)

In [29]:
data = np.loadtxt('data/hw7/in.dta.txt')

N_train = 10
X_train = data[-N_train:,:-1]
Y_train = data[-N_train:,2]

N_val = 25
X_val = data[:N_val,:-1]
Y_val = data[:N_val,2]

def phi(X, k):
    assert(X.shape == (2,))
    x1, x2 = X
    return np.array([1, x1, x2, x1 * x1, x2 * x2, x1 * x2, np.abs(x1-x2), np.abs(x1+x2)])[:k+1]

for k in range(3, 8):
    # Non-Linear Transformation
    Z_train = np.apply_along_axis(phi, 1, X_train, k)
    Z_val = np.apply_along_axis(phi, 1, X_val, k)

    # Linear Regression
    X_dagger = np.dot(np.linalg.inv(np.dot(Z_train.T, Z_train)), Z_train.T)
    W = np.dot(X_dagger, Y_train)

    # Validation Error
    preds_val = np.sign(np.dot(Z_val, W))
    E_VAL = sum(preds_val != Y_val) / N_val

    print(f"E_VAL for k={k} is {E_VAL}")

E_VAL for k=3 is 0.28
E_VAL for k=4 is 0.36
E_VAL for k=5 is 0.2
E_VAL for k=6 is 0.08
E_VAL for k=7 is 0.12


So the answer is still **[d]** as above, not too much relative change here compared to 1. except that the predicted errors are much higher (as expected).

![](imgs/val4.png)

In [30]:
test = np.loadtxt('data/hw7/out.dta.txt')

X_test = test[:,:-1]
Y_test = test[:,2]
N_test = Y_test.shape[0]

for k in range(3, 8):
    # Non-Linear Transformation
    Z_train = np.apply_along_axis(phi, 1, X_train, k)
    Z_test = np.apply_along_axis(phi, 1, X_test, k)

    # Linear Regression
    X_dagger = np.dot(np.linalg.inv(np.dot(Z_train.T, Z_train)), Z_train.T)
    W = np.dot(X_dagger, Y_train)

    # Out of Sample Error
    preds_test = np.sign(np.dot(Z_test, W))
    E_OUT = sum(preds_test != Y_test) / N_test

    print(f"E_OUT for k={k} is {E_OUT}")

E_OUT for k=3 is 0.396
E_OUT for k=4 is 0.388
E_OUT for k=5 is 0.284
E_OUT for k=6 is 0.192
E_OUT for k=7 is 0.196


And indeed the prediction is better this time! **[d]** as well.

![](imgs/val5.png)

For the model (d) chosen in 1., we had

E_OUT for k=6 is `0.084`

and for the model (d) chosen in 3., we had 

E_OUT for k=6 is `0.192`

That's closest to answer **[b]**

## Validation Bias

![](imgs/valbias1.png)

Stealing the result and approach from [here](https://math.stackexchange.com/questions/786392/expectation-of-minimum-of-n-i-i-d-uniform-random-variables),
the expected value of $min(e1, e2)$ is $\frac{1}{n+1}=\frac{1}{3}$, hence the closest answer should be **[d]**

## Cross Validation

![](imgs/xvalidation.png)

In [3]:
P = {
    "a": np.sqrt(np.sqrt(3)+4),
    "b": np.sqrt(np.sqrt(3)-1),
    "c": np.sqrt(9 + 4 * np.sqrt(6)),
    "d": np.sqrt(9 - np.sqrt(6))
}

# Check all 4 answers [a], [b], [c] and [d]
for pName, p in P.items():
    X = np.array([-1, p, 1])
    Y = np.array([0, 1, 0])
    N = Y.shape[0]

    # Leave-one-out X-Validation
    E_VALS_CONST = []
    E_VALS_LIN = []
    for i in range(N):
        X_train = np.delete(X, i)[:, None]
        Y_train = np.delete(Y, i)[:, None]
        N_train = Y_train.shape[0]

        X_val = X[i]
        Y_val = Y[i]
        N_val = 1

        # Constant Model
        b = sum(Y_train) / N_train
        # In-sample Error
        preds = np.repeat(b, N_train)
        E_IN = (1/N_train) * sum(np.power(Y_train[n] - preds[n], 2) for n in range(N_train))
        # Validation Error
        pred_val = b
        E_VALS_CONST.append(np.power(Y_val - pred_val, 2))

        # Linear Model
        X_train_with_const = np.c_[np.ones(N_train), X_train]
        X_val_with_const = [1, X_val]
        X_dagger = np.dot(np.linalg.inv(np.dot(X_train_with_const.T, X_train_with_const)), X_train_with_const.T)
        W = np.dot(X_dagger, Y_train)
        # In-sample Error
        preds = np.dot(X_train_with_const, W)
        E_IN = (1/N_train) * sum(np.power(Y_train[n] - preds[n], 2) for n in range(N_train))
        # Validation Error
        pred_val = np.dot(X_val_with_const, W)
        E_VALS_LIN.append(np.power(Y_val - pred_val, 2))



    print(f"Average validation squared error for the constant model using leave-one-out crossvalidation for answer [{pName}] is {sum(E_VALS_CONST)/len(E_VALS_CONST)}")
    print(f"Average validation squared error for the linear model using leave-one-out crossvalidation for answer [{pName}] is {sum(E_VALS_LIN)/len(E_VALS_LIN)}")


Average validation squared error for the constant model using leave-one-out crossvalidation for answer [a] is [0.5]
Average validation squared error for the linear model using leave-one-out crossvalidation for answer [a] is [1.13504337]
Average validation squared error for the constant model using leave-one-out crossvalidation for answer [b] is [0.5]
Average validation squared error for the linear model using leave-one-out crossvalidation for answer [b] is [64.66494841]
Average validation squared error for the constant model using leave-one-out crossvalidation for answer [c] is [0.5]
Average validation squared error for the linear model using leave-one-out crossvalidation for answer [c] is [0.5]
Average validation squared error for the constant model using leave-one-out crossvalidation for answer [d] is [0.5]
Average validation squared error for the linear model using leave-one-out crossvalidation for answer [d] is [0.98688393]


So the answer is **[c]**

## PLA vs. SVM

![](imgs/svm1.png)

In [12]:
%%time

EXPERIMENTS = 1000
N = 10

def generate_data(N):
    # Generate a target function (see HW1 for explanations)
    x1 = np.random.uniform(-1, 1, 2)
    x2 = np.random.uniform(-1, 1, 2)
    m = (x1[1] - x2[1]) / (x1[0] - x2[0])
    c = x1[1] - m*x1[0]
    def y(X):
        x, y = X
        return 1 if y > m*x+c else -1
    
    X = np.c_[np.random.uniform(-1, 1, N), np.random.uniform(-1, 1, N)]
    Y = np.apply_along_axis(y, 1, X)
    # Generate a new dataset if all points fall on the same side of the line
    while len(set(Y)) == 1:
        X = np.c_[np.random.uniform(-1, 1, N), np.random.uniform(-1, 1, N)]
        Y = np.apply_along_axis(y, 1, X)
        
    return (X, Y, y)

def generate_test_data(N, y):
    X = np.c_[np.random.uniform(-1, 1, N), np.random.uniform(-1, 1, N)]
    Y = np.apply_along_axis(y, 1, X)
    
    return (X, Y)

def perceptron(N): 
    X, Y, y = generate_data(N)
    clf = Perceptron(tol=None)
    clf.fit(X, Y)
    X_test, Y_test = generate_test_data(100*N, y)
    return clf.score(X_test, Y_test)

def SVM(N): 
    X, Y, y = generate_data(N)
    clf = svm.SVC(kernel='linear', C=np.Inf, cache_size=20000)
    clf.fit(X, Y)
    X_test, Y_test = generate_test_data(100*N, y)
    return clf.score(X_test, Y_test)

# How often does SVM() outperform perceptron()?
score = functools.reduce(lambda score, _: score + ((1/EXPERIMENTS) if SVM(N) > perceptron(N) else 0), range(EXPERIMENTS), 0)

print(f"Average % where g_SVM outperforms g_PLA is {score}")


Average % where g_SVM outperforms g_PLA is 0.5720000000000004
CPU times: user 7.75 s, sys: 2.01 ms, total: 7.75 s
Wall time: 7.75 s


That's closest to 60%, or **[c]**, so even on 10 data points, SVM outperforms PLA here!

![](imgs/svm2.png)

In [13]:
N = 100

# How often does SVM() outperform perceptron()?
score = functools.reduce(lambda score, _: score + ((1/EXPERIMENTS) if SVM(N) > perceptron(N) else 0), range(EXPERIMENTS), 0)

print(f"Average % where g_SVM outperforms g_PLA is {score}")


Average % where g_SVM outperforms g_PLA is 0.5680000000000004


That's actually closer to **[c]** than the correct answer **[d]**, so the algorithm impls must be different here.

Indeed, `sklearn.svm.SVC` seems to be solving the following problem: 

![](imgs/sklearn.png)

And I'm not sure how to get rid of the zeta terms...

![](imgs/svm3.png)

In [17]:
EXPERIMENTS = 1000
N = 100

def SVM(N): 
    X, Y, y = generate_data(N)
    clf = svm.SVC(C=np.Inf, kernel='linear', cache_size=20000)
    clf.fit(X, Y)
    X_test, Y_test = generate_test_data(100*N, y)
    return (clf.score(X_test, Y_test), sum(clf.n_support_))

# How often does SVM() outperform perceptron()?
score = functools.reduce(lambda score, _: score + SVM(N)[1]/EXPERIMENTS, range(EXPERIMENTS), 0)

print(f"Average # of support vectors of g_SVM is {score}")


Average # of support vectors of g_SVM is 2.998000000000002


**[b]**